In [8]:
from influxdb import InfluxDBClient
import socket
import datetime
ip = '127.0.0.1'
port_socket = 5001
port_influx = 8086
timeout = 5
data_size = 1024
name_database = 'work log'
client = InfluxDBClient(host = ip, port = 8086)
client.create_database(name_database)
client.switch_database('work log')
with socket.socket() as sock:
    sock.bind((ip, port_socket))
    sock.listen()
    while True:
        conn, addr, = sock.accept()
        conn.settimeout(timeout)
        with conn:
            try:
                data = str(conn.recv(data_size)).split(';')
                camera_id = data[0]
                weather = data[1]
                area = data[2]
                data_body = [
                    {
                        "measurement": "camera",
                        "tags": {
                            "Id": camera_id
                        },
                        "time": datetime.datetime.now(),
                        "fields": {
                            "area": area
                            "weather": weather
                        }
                    }
                ]
                print (data_body)
                client.write_points(data_body)
            except:
                print('Error reading data')
                break